# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df= pd.read_csv('../output_data/weather_df.csv')
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Wenling,28.3669,121.3600,78.91,94,100,8.68,CN,1627672404
1,Shingū,33.7333,135.9833,77.92,86,16,4.07,JP,1627672404
2,Adrar,20.5022,-10.0711,101.84,16,36,5.32,MR,1627672405
3,East London,-33.0153,27.9116,60.24,72,12,6.42,ZA,1627672405
4,Kruisfontein,-34.0033,24.7314,49.89,79,26,1.97,ZA,1627672405
...,...,...,...,...,...,...,...,...,...
537,Santa Maria,-29.6842,-53.8069,56.23,51,0,5.37,BR,1627672575
538,Acajutla,13.5928,-89.8275,95.34,66,42,7.11,SV,1627672576
539,Alghero,40.5589,8.3181,82.99,89,0,9.22,IT,1627672576
540,Haines Junction,60.7522,-137.5108,66.15,45,0,2.30,CA,1627672576


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations= weather_df[['Lat','Lng']].astype(float)

# Get the humidity rating
humidity= weather_df['Humidity'].astype(float)

# Plot Heatmap, create a gmaps figure 
fig= gmaps.figure()

#create a heat layer (search the gmaps documentation - hint there's a heatmap_layer method)
heat_layer= gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)

#add the layer to your figure
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = weather_df[(weather_df['Max Temp'] >70) &
                      (weather_df['Max Temp'] <80) &
                      (weather_df['Wind Speed'] <10) &
                      (weather_df['Cloudiness'] ==0)
                     ]
ideal_df.dropna(inplace=True)
ideal_df

<ipython-input-5-af4799a88c43>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df.dropna(inplace=True)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
245,Constanța,44.1833,28.6500,74.08,83,0,8.05,RO,1627672195
280,Heraclea,37.5133,34.0467,74.28,33,0,8.99,TR,1627672488
311,Saint-François,46.4154,3.9054,72.84,70,0,3.44,FR,1627672496
317,Lucapa,-8.4192,20.7447,74.37,27,0,3.62,AO,1627672498
354,Divnomorskoye,44.5011,38.1353,74.70,72,0,4.88,RU,1627672517
360,Illapel,-31.6308,-71.1653,74.26,22,0,5.19,CL,1627672519
379,Hasaki,35.7333,140.8333,73.80,85,0,8.95,JP,1627672527
432,Benguela,-12.5763,13.4055,71.37,85,0,6.26,AO,1627672543
435,Lagoa,39.0500,-27.9833,71.15,81,0,7.81,PT,1627672544
466,Yatou,37.1500,122.3833,76.37,91,0,8.99,CN,1627672553


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-6-245266804107>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
245,Constanța,44.1833,28.6500,74.08,83,0,8.05,RO,1627672195,
280,Heraclea,37.5133,34.0467,74.28,33,0,8.99,TR,1627672488,
311,Saint-François,46.4154,3.9054,72.84,70,0,3.44,FR,1627672496,
317,Lucapa,-8.4192,20.7447,74.37,27,0,3.62,AO,1627672498,
354,Divnomorskoye,44.5011,38.1353,74.70,72,0,4.88,RU,1627672517,
360,Illapel,-31.6308,-71.1653,74.26,22,0,5.19,CL,1627672519,
379,Hasaki,35.7333,140.8333,73.80,85,0,8.95,JP,1627672527,
432,Benguela,-12.5763,13.4055,71.37,85,0,6.26,AO,1627672543,
435,Lagoa,39.0500,-27.9833,71.15,81,0,7.81,PT,1627672544,
466,Yatou,37.1500,122.3833,76.37,91,0,8.99,CN,1627672553,


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name


In [18]:
# Add marker layer ontop of heat map

# Display figure
